In [2]:
import joblib
import weaviate
from weaviate.classes.query import (
    Filter,
    Rerank
)
%load_ext autoreload
%autoreload 2

In [3]:
bbc_data = joblib.load('bbc_data.joblib')

In [4]:
from utils import print_object_properties

print_object_properties(bbc_data[0:3])

article_content: Justin Welby speaks on BBC Radio 4's Today programme as part of a special show guest edited by Dame ...(truncated)
description: The Archbishop of Canterbury urges politicians to "forswear wedge issues" and avoid divisive topics.
guid: https://www.bbc.co.uk/news/uk-67844356
link: https://www.bbc.co.uk/news/uk-67844356?at_medium=RSS&at_campaign=KARANGA
pubDate: 2024-01-01 00:00:04
title: Justin Welby: Political leaders should treat opponents as human beings

article_content: Rishi Sunak at a lung cancer screening programme in Nottingham in June 2023 Almost three million peo...(truncated)
description: Record numbers are being tested for cancer but treatment targets are still being missed in England.
guid: https://www.bbc.co.uk/news/health-67841348
link: https://www.bbc.co.uk/news/health-67841348?at_medium=RSS&at_campaign=KARANGA
pubDate: 2024-01-01 00:09:56
title: Almost three million tested for cancer in England

article_content: An increase in the cost of domestic energ

In [1]:
from google.colab import userdata
weaviate_api_key = userdata.get('WEAVIATE_API')
weaviate_url = "de9kmol9ty2wenasbrsvq.c0.asia-southeast1.gcp.weaviate.cloud"


In [6]:
from weaviate.classes.init import Auth
from weaviate.classes.config import Configure
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

In [7]:
collection = client.collections.create(
    name="bbc_news",
    vector_config=Configure.Vectors.text2vec_weaviate(),
    generative_config=Configure.Generative.cohere()
    )

In [8]:
with collection.batch.dynamic() as batch:
    for i, data_object in enumerate(bbc_data):
        batch.add_object(
            properties=data_object,
        )

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/weaviate/warnings.py:256: UserWarning: Con002: You are using the datetime object 2024-11-01 17:13:10 without a timezone. The timezone will be set to UTC.
            To use a different timezone, specify it in the datetime object. For example:
            datetime.datetime(2021, 1, 1, 0, 0, 0, tzinfo=datetime.timezone(-datetime.timedelta(hours=2))).isoformat() = 2021-01-01T00:00:00-02:00
            
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/weaviate/warnings.py:256: UserWarning: Con002: You are using the datetime object 2024-11-01 22:26:47 without a timezone. The timezone will be set to UTC.
            To use a different timezone, specify it in the datetime object. For example:
            datetime.datetime(2021, 1, 1, 0, 0, 0, tzinfo=datetime.timezone(-datetime.timedelta(hours=2))).isoformat() = 2021-01-01T00:00:00-02:00
            
  warnings.warn(
/usr/local/lib/python3.11/dis

In [9]:
len(collection)

9973

In [19]:
obj = collection.query.fetch_objects(limit=1, include_vector=True).objects[0]

In [20]:
print_object_properties(obj.properties)

article_content: The MP behind a bid to legalise assisted dying in England and Wales has argued it would help stop pe...(truncated)
description: Adults expected to die within six months would be eligible under proposals for England and Wales in a bill which has just been published.
guid: https://www.bbc.com/news/articles/c154pwlv4epo#0
link: https://www.bbc.com/news/articles/c154pwlv4epo
pubDate: 2024-11-11 23:19:46+00:00
title: Assisted dying bill has strict safeguards, says MP proposing law change



In [26]:
obj.vector['default'][0:15]

[0.024595895782113075,
 0.043929412961006165,
 0.017777685075998306,
 -0.018578480929136276,
 -0.014780418016016483,
 -0.07019554078578949,
 0.06685508042573929,
 -0.016313370317220688,
 -0.0016716629033908248,
 0.0830082893371582,
 -0.006097494158893824,
 0.03647056594491005,
 0.1331610381603241,
 0.016770970076322556,
 -0.0050478787161409855]

In [27]:
len(obj.vector['default'])

1024

In [51]:
def filtered_by_metadata(metadata_property: str, values: list[str], collection:"weaviate.collections.collection.sync.Collection", limit:int=5)->list:
  response = collection.query.fetch_objects(limit=limit,filters=Filter.by_property(metadata_property).contains_any(values))
  response_objects = [x.properties for x in response.objects]
  return response_objects

In [52]:
fiter = filtered_by_metadata('title', ['Taylor Swift'], collection, limit=2)

In [54]:
print_object_properties(fiter)

article_content: The 2024 awards season kicked off in style at the Golden Globes - the first major red carpet event o...(truncated)
description: Stars including Margot Robbie and Taylor Swift arrived in a variety of eye-catching outfits.
guid: https://www.bbc.co.uk/news/entertainment-arts-67908727
link: https://www.bbc.co.uk/news/entertainment-arts-67908727?at_medium=RSS&at_campaign=KARANGA
pubDate: 2024-01-08 03:23:58+00:00
title: Margot Robbie, Taylor Swift and more on Golden Globes red carpet

article_content: TikTok has removed music by stars including Taylor Swift, The Weeknd and Olivia Rodrigo from videos ...(truncated)
description: It comes after a licensing rights row between the platform and the artists' record label, Universal.
guid: https://www.bbc.co.uk/news/technology-68166028
link: https://www.bbc.co.uk/news/technology-68166028?at_medium=RSS&at_campaign=KARANGA
pubDate: 2024-02-01 17:17:45+00:00
title: TikTok mutes users' videos as it pulls Taylor Swift and The Weeknd's m

In [59]:
def sementic_search_retrieve(query: str, collection: "weaviate.collections.collection.sync.Collection", top_k:int=5)->list:
  response = collection.query.near_text(query=query,  limit=top_k)
  response_objects = [x.properties for x in response.objects]
  return response_objects


In [60]:
sementic_search = sementic_search_retrieve(query = 'Tell me about the last Taylor Swift show', collection = collection, top_k = 2)

In [61]:
print_object_properties(sementic_search)

article_content: Taylor Swift was joined by "best friend" Ed Sheeran as she kicked off the final part of the European...(truncated)
description: Swift closes the European leg of her tour with five record-breaking performances at Wembley Stadium.
guid: https://www.bbc.com/news/articles/cwyw74p8ejeo#0
link: https://www.bbc.com/news/articles/cwyw74p8ejeo
pubDate: 2024-08-15 21:27:53+00:00
title: Ed Sheeran joins Taylor Swift for Eras Tour return

article_content: Taylor Swift has finished the European leg of her Eras Tour with a record-breaking show at Wembley S...(truncated)
description: The star is joined by Florence + The Machine and sings So Long, London at her final UK show.
guid: https://www.bbc.com/news/articles/cr5nr3n6epvo#2
link: https://www.bbc.com/news/articles/cr5nr3n6epvo
pubDate: 2024-08-21 03:02:08+00:00
title: 'I've never had it this good' - Taylor Swift thanks fans after new Wembley record




In [62]:
def bm25_retrieve(query: str, collection: "weaviate.collections.collection.sync.Collection", top_k:int=5)->list:
  response = collection.query.bm25(query=query, limit=top_k)
  response_objects = [x.properties for x in response.objects]
  return response_objects

In [63]:
keyword_search = bm25_retrieve(query = 'Tell me about the last Taylor Swift show', collection = collection, top_k = 2)

In [65]:
print_object_properties(keyword_search)

article_content: There was a very special guest at Taylor Swift's most recent London show - the singer's boyfriend Tr...(truncated)
description: Kelce joined Swift on stage as a dancer for I Can Do It With a Broken Heart. 
guid: https://www.bbc.com/news/articles/c2999rpn6e5o#6
link: https://www.bbc.com/news/articles/c2999rpn6e5o
pubDate: 2024-06-24 07:31:17+00:00
title: Kelce makes surprise appearance during Taylor Swift show

article_content: The Tortured Poets Department, released on Friday, features 31 songs For two female journalists in t...(truncated)
description: From exes who wasted our time, to comfort-eating after a breakup. We've all been there and so has Swift.
guid: https://www.bbc.co.uk/news/entertainment-arts-68856232#1
link: https://www.bbc.co.uk/news/entertainment-arts-68856232
pubDate: 2024-04-20 15:36:19+00:00
title: How Taylor Swift captured modern dating despair




In [70]:
def hybrid_retrieve(query: str, collection: "weaviate.collections.collection.sync.Collection", alpha: float=0.5, top_k:int=5)->list:
  response = collection.query.hybrid(query=query, alpha=alpha, limit=top_k)
  response_objects = [x.properties for x in response.objects]
  return response_objects

In [71]:
  print_object_properties(hybrid_retrieve('Tell me about the last Taylor Swift show', collection, top_k = 2))

article_content: Taylor Swift was joined by "best friend" Ed Sheeran as she kicked off the final part of the European...(truncated)
description: Swift closes the European leg of her tour with five record-breaking performances at Wembley Stadium.
guid: https://www.bbc.com/news/articles/cwyw74p8ejeo#0
link: https://www.bbc.com/news/articles/cwyw74p8ejeo
pubDate: 2024-08-15 21:27:53+00:00
title: Ed Sheeran joins Taylor Swift for Eras Tour return

article_content: Taylor Swift has finished the European leg of her Eras Tour with a record-breaking show at Wembley S...(truncated)
description: The star is joined by Florence + The Machine and sings So Long, London at her final UK show.
guid: https://www.bbc.com/news/articles/cr5nr3n6epvo#2
link: https://www.bbc.com/news/articles/cr5nr3n6epvo
pubDate: 2024-08-21 03:02:08+00:00
title: 'I've never had it this good' - Taylor Swift thanks fans after new Wembley record




In [76]:
# from weaviate.classes.query import Rerank

# def sementic_search_with_reranking(query: str, rerank_property:str, collection:"weaviate.collections.collection.sync.Collection",rerank_query: str=None,top_k:int=5)->list:
#   if rerank_query is None:
#     rerank_query = query

#   rerank=Rerank(
#         prop=rerank_property,
#         query=rerank_query
#     )

#   response = collection.query.near_text(query=query, limit=top_k, rerank=rerank)

#   response_objects = [ x.properties for x in response.ojects]

#   return response_objects


In [77]:
# query = 'Tell me about the conflicts in Latin America'
# results = sementic_search_with_reranking(query, collection = collection, top_k = 2, rerank_property = 'article_content')

In [81]:
def generate_final_prompt(query: str, top_k: int, retrieve_function: callable, rerank_query: str = None,rerank_property: str = None, use_rerank: bool = False, use_rag: bool = True) -> list:
  if not use_rag:
    return query

  if not use_rerank:
    response = retrieve_function(query, top_k=top_k, collection=collection)
  else:
    pass


  formatted_text = ""

  for document in response:
     document_layout = (
            f"Title: {document['title']}, Chunk: {document['description']}, "
            f"Published at: {document['pubDate']}\nURL: {document['link']}"
        )

     formatted_text += document_layout + "\n"

  retrieve_data_formatted = formatted_text

  prompt = (
       f"Answer the user query below. There will be provided additional information for you to compose your answer. "
       f"The relevant information provided is from 2024 and it should be added as your overall knowledge to answer the query, "
       f"you should not rely only on this information to answer the query, but add it to your overall knowledge."
       f"The news data is ordered by relevance."
       f"Query: {query}\n"
       f"2024 News: {retrieve_data_formatted}"
  )

  return prompt

In [82]:
prompt = generate_final_prompt("Tell me the economic situation of the US in 2024.", top_k = 5, retrieve_function = sementic_search_retrieve, use_rerank = False, rerank_property = 'title')

In [88]:
from langchain_groq import ChatGroq
import langchain


groq_api_key = userdata.get('GROQ_API')

llm = ChatGroq(api_key=groq_api_key, model="llama3-8b-8192", temperature=0.5)
response = llm.invoke(prompt)

In [90]:
print(response.content)

Based on the provided news articles from 2024, here's an overview of the economic situation in the US:

As of 2024, the US economy is experiencing a boom, which has consequences for the global balance of power and the UK. Inflation has decreased, and the "vibes" are up, but younger Americans remain sceptical due to the pain of soaring prices. This is reflected in the fact that US voters feel better about the economy, which may help President Biden.

However, it's essential to note that the economic situation is complex and influenced by various factors. While the US economy is booming, the UK economy is facing challenges, with Chancellor Jeremy Hunt comparing it to other countries and acknowledging that it has shrunk for two consecutive quarters.

It's also worth mentioning that the definition of a recession is when the economy has shrunk for two or three consecutive quarters. This means that the US economy is not currently in a recession, but it's essential to monitor the situation cl